In [1]:
from omegaconf import OmegaConf
from lerobot.common.policies.factory import make_policy, _policy_cfg_from_hydra_cfg
from pathlib import Path
from lerobot.common.envs.factory import make_env
from importlib import import_module
from safetensors import safe_open
from lerobot.scripts.eval import eval_policy
import torch
torch.set_grad_enabled(False)

def load_safetensors(path):
    tensors = {}
    with safe_open(path, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)
    return tensors

device = 0

In [2]:
# task = 'insertion'
task = 'transfer_cube'

In [3]:
config_path = './configs/arp.yaml'
ckpt_path = f'./weights/model.{task}.safetensors'

In [4]:
cfg = OmegaConf.load(config_path)
cfg['dataset_repo_id'] = f'lerobot/aloha_sim_{task}_human'
if task == 'insertion':
    cfg.env.task = 'AlohaInsertion-v0'
else:
    cfg.env.task = 'AlohaTransferCube-v0'

In [5]:
prefix = 'lerobot.common.policies.' + cfg.policy['name']
print(f'importing policy module from: {prefix}')
config_mod = import_module(prefix + '.configuration')
Config = config_mod.ARPConfig if hasattr(config_mod, 'ARPConfig') else config_mod.Config
modeling_mod = import_module(prefix + '.modeling')
Policy = modeling_mod.ARPPolicy if hasattr(modeling_mod, 'ARPPolicy') else modeling_mod.Policy

importing policy module from: lerobot.common.policies.autoregressive_policy


In [6]:
config = _policy_cfg_from_hydra_cfg(Config, cfg)

In [8]:
policy = Policy(config)
policy.load_state_dict(load_safetensors(ckpt_path))

<All keys matched successfully>

In [9]:
policy = policy.to(device)

In [10]:
env = make_env(cfg, 1)

In [11]:
eval_policy(env, policy, 30, max_episodes_rendered=30, videos_dir=Path('./outputs/demo/' + cfg['dataset_repo_id']), 
            enable_progbar=True, enable_inner_progbar=True)

Stepping through eval batches:  73%|███████▎  | 22/30 [35:39<12:58, 97.26s/it, running_success_rate=81.8%]


KeyboardInterrupt: 